In [21]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from keras import models, layers, optimizers
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
import os

In [77]:
Images = [] 
Labels = []  # 0 for Building , 1 for forest, 2 for glacier, 3 for mountain, 4 for Sea , 5 for Street 
label = 0 

directory = '/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/'

for labels in os.listdir(directory): #Main Directory where each class label is present as folder name. 
    if labels == 'tallbuilding': #Folder contain Glacier Images get the '2' class label. 
        label = 0 
    elif labels == 'suburb': 
        label = 1 
    elif labels == 'opencountry': 
        label = 2 
    elif labels == 'forest': 
        label = 3 
    elif labels == 'street': 
        label = 4 
    elif labels == 'mountain': 
        label = 5
    elif labels == 'office': 
        label = 6
    elif labels == 'livingroom': 
        label = 7
    elif labels == 'kitchen': 
        label = 8
    elif labels == 'insidecity': 
        label = 9
    elif labels == 'highway': 
        label = 10
    elif labels == 'coast': 
        label = 11
    elif labels == 'bedroom': 
        label = 12


    for image_file in os.listdir(directory+labels): #Extracting the file name of the image from Class Label folder 
        image = cv2.imread(directory+labels+r'/'+image_file) #Reading the image (OpenCV) 
        image = cv2.resize(image,(224,224),interpolation=cv2.INTER_AREA) #Resize the image, Some images are different sizes. (Resizing is very Important) 
        Images.append(image) 
        Labels.append(label) 
     
#shuffle(Images,Labels,random_state=5) #Shuffle the dataset you just prepared. 
 

def get_classlabel(class_code): 
    labels = {0:'tallbuilding', 1:'suburb', 2:'opencountry', 3:'forest', 4:'street', 5:'mountain',6:'office',7:'livingroom', 8:'kitchen', 9:'insidecity',10:'highway',11:'coast',12:'bedroom'}
 
    return labels[class_code] 

Images = np.array(Images)
Images = Images.astype('float32')
Images.shape

(2819, 224, 224, 3)

In [78]:
from keras.utils import to_categorical
from keras import utils as np_utils
Labels = np.array(Labels)
Labels = np_utils.to_categorical(Labels)
Labels.shape

(2819, 13)

In [55]:

batch_size = 32

In [79]:
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(Images, Labels, test_size=0.2, shuffle= True)

In [80]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        shear_range=0.4,
        zoom_range=0.4,
        rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.4,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.4,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size=batch_size)

val_generator = val_datagen.flow(
    x_valid,
    y_valid,
    batch_size=batch_size)
    
 

In [58]:
from keras.callbacks import LearningRateScheduler
import math
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

In [81]:
del Images
del Labels
import gc
gc.collect()

26

**NASNetMobile**

In [97]:
from keras import backend as K
from keras import Model
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
#import inception with pre-trained weights. do not include fully #connected layers


K.set_learning_phase(0)

base_model = applications.nasnet.NASNetMobile(weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False

K.set_learning_phase(1)

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = layers.Dense(1024, activation='relu')(x)
# and a fully connected output/classification layer
predictions = layers.Dense(13, activation='softmax')(x)


#combine base and top model
fullModel = Model(inputs= base_model.input, outputs= predictions)

fullModel.compile(loss='categorical_crossentropy',
             optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), 
             metrics=['accuracy'])

19996672/19993432 [==============================] - 0s 0us/step


**Xception**

In [42]:
from keras import Model
from keras.applications import Xception
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
base_model = Xception(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(13, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# fix the feature extraction part of the model
for layer in base_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False
        
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

**InceptionV3**

In [29]:
from keras import backend as K
from keras import Model
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
#import inception with pre-trained weights. do not include fully #connected layers


K.set_learning_phase(0)

base_model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False

K.set_learning_phase(1)

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = layers.Dense(1024, activation='relu')(x)
# and a fully connected output/classification layer
predictions = layers.Dense(13, activation='softmax')(x)


#combine base and top model
fullModel = Model(inputs= base_model.input, outputs= predictions)

fullModel.compile(loss='categorical_crossentropy',
             optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), 
             metrics=['accuracy'])

**DenseNet201**

In [82]:
from keras import Model
from keras.applications import Xception
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
#base_model = applications.densenet.DenseNet121(weights='imagenet', include_top=False)
base_model = applications.densenet.DenseNet201(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(13, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# fix the feature extraction part of the model
for layer in base_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False
        
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

74842112/74836368 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
___________________________

**EfficientNetB3**

In [60]:
!git clone https://github.com/qubvel/efficientnet.git

Cloning into 'efficientnet'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 280 (delta 11), reused 2 (delta 2), pack-reused 254
Receiving objects: 100% (280/280), 833.12 KiB | 0 bytes/s, done.
Resolving deltas: 100% (151/151), done.


In [61]:
import efficientnet.efficientnet.keras as efn
from keras import Model
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
base_model = efn.EfficientNetB2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(13, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# fix the feature extraction part of the model
for layer in base_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False
        
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

31940608/31936256 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 3 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 3 0           stem_bn[0][0]                    
___________________________

In [100]:
history = fullModel.fit_generator(generator=train_generator,
                              steps_per_epoch=len(x_train) // batch_size + 1 ,  # added in Kaggle
                              epochs=50,
                              validation_data=val_generator,
                              validation_steps=len(x_valid) // batch_size + 1 , # added in Kaggle
                              callbacks=callbacks_list
                             )

Epoch 1/50
71/71 [==============================] - 23s 330ms/step - loss: 0.5484 - accuracy: 0.8013 - val_loss: 0.6108 - val_accuracy: 0.8387
Epoch 2/50
71/71 [==============================] - 23s 321ms/step - loss: 0.5884 - accuracy: 0.7973 - val_loss: 0.7189 - val_accuracy: 0.8191
Epoch 3/50
71/71 [==============================] - 23s 321ms/step - loss: 0.5572 - accuracy: 0.8040 - val_loss: 0.5460 - val_accuracy: 0.8191
Epoch 4/50
71/71 [==============================] - 23s 322ms/step - loss: 0.5757 - accuracy: 0.7933 - val_loss: 0.2298 - val_accuracy: 0.8652
Epoch 5/50
71/71 [==============================] - 23s 327ms/step - loss: 0.5506 - accuracy: 0.8044 - val_loss: 0.2937 - val_accuracy: 0.8475
Epoch 6/50
71/71 [==============================] - 23s 324ms/step - loss: 0.5609 - accuracy: 0.7965 - val_loss: 0.2721 - val_accuracy: 0.8351
Epoch 7/50
71/71 [==============================] - 23s 322ms/step - loss: 0.5465 - accuracy: 0.7982 - val_loss: 0.3542 - val_accuracy: 0.8404

KeyboardInterrupt: 

In [ ]:
for layer in model.layers:
    layer.trainable = True
    
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=len(x_train) // batch_size + 1 ,  # added in Kaggle
                              epochs=50,
                              validation_data=val_generator,
                              validation_steps=len(x_valid) // batch_size + 1  # added in Kaggle
                             )

In [84]:
import glob
TestImages = []
TestLabel = []
for imagepath in sorted(glob.glob("/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/*.jpg")):
        img=cv2.imread(imagepath)
        #img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                         
        img=cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA)
        TestImages.append(img)
        TestLabel.append(imagepath[-14:-4])

In [85]:
TestImages = np.array(TestImages)
TestImages = TestImages.astype('float32')
TestImages = TestImages/255
TestImages.shape

(1040, 224, 224, 3)

In [102]:
pred5=s=fullModel.predict(TestImages)
predict_digit=np.argmax(pred5,axis=1)
i = 0
for i in predict_digit:
    print(get_classlabel(i))

highway
bedroom
suburb
bedroom
tallbuilding
insidecity
mountain
forest
coast
street
highway
bedroom
mountain
street
insidecity
coast
highway
livingroom
kitchen
insidecity
highway
mountain
highway
insidecity
office
highway
forest
bedroom
mountain
forest
street
forest
bedroom
highway
street
suburb
street
coast
forest
suburb
suburb
bedroom
suburb
bedroom
livingroom
street
coast
insidecity
highway
tallbuilding
kitchen
office
kitchen
mountain
suburb
bedroom
tallbuilding
highway
street
kitchen
forest
bedroom
insidecity
bedroom
opencountry
office
opencountry
tallbuilding
kitchen
bedroom
insidecity
highway
bedroom
livingroom
kitchen
coast
insidecity
coast
opencountry
opencountry
mountain
forest
livingroom
street
livingroom
mountain
office
forest
kitchen
street
suburb
street
livingroom
suburb
opencountry
livingroom
insidecity
insidecity
livingroom
bedroom
highway
coast
tallbuilding
tallbuilding
kitchen
opencountry
mountain
opencountry
bedroom
office
livingroom
forest
livingroom
bedroom
highway


In [103]:
predall=pred1+pred2+pred3+pred4+pred5

In [104]:
predict_digit=np.argmax(predall,axis=1)
i = 0
for i in predict_digit:
    print(get_classlabel(i))

highway
bedroom
suburb
bedroom
tallbuilding
insidecity
coast
forest
coast
street
highway
bedroom
mountain
street
insidecity
coast
highway
livingroom
kitchen
insidecity
highway
mountain
highway
insidecity
office
highway
forest
bedroom
mountain
forest
street
forest
bedroom
highway
highway
suburb
street
coast
forest
suburb
suburb
bedroom
suburb
livingroom
livingroom
street
coast
insidecity
street
tallbuilding
kitchen
office
livingroom
mountain
suburb
bedroom
tallbuilding
highway
street
kitchen
forest
bedroom
suburb
bedroom
opencountry
office
opencountry
tallbuilding
kitchen
bedroom
insidecity
highway
bedroom
bedroom
kitchen
coast
insidecity
coast
opencountry
opencountry
mountain
forest
livingroom
street
livingroom
mountain
office
forest
kitchen
street
suburb
street
office
suburb
opencountry
livingroom
street
insidecity
office
bedroom
highway
coast
tallbuilding
tallbuilding
kitchen
mountain
opencountry
opencountry
bedroom
office
livingroom
forest
livingroom
bedroom
coast
street
livingroom


In [105]:
predall_weight = (pred1*0.8)+pred2+pred3+(pred4*0.8)+(pred5*0.6)

In [106]:
predict_digit=np.argmax(predall_weight,axis=1)
i = 0
for i in predict_digit:
    print(get_classlabel(i))

highway
bedroom
suburb
bedroom
tallbuilding
insidecity
coast
forest
coast
street
highway
bedroom
mountain
street
insidecity
coast
highway
livingroom
kitchen
insidecity
highway
mountain
highway
insidecity
office
highway
forest
bedroom
mountain
forest
street
forest
bedroom
highway
highway
suburb
street
coast
forest
suburb
suburb
bedroom
suburb
livingroom
livingroom
street
coast
insidecity
street
tallbuilding
kitchen
office
livingroom
mountain
suburb
bedroom
tallbuilding
highway
street
kitchen
forest
bedroom
suburb
bedroom
coast
office
opencountry
highway
kitchen
bedroom
insidecity
highway
bedroom
bedroom
kitchen
coast
insidecity
coast
opencountry
opencountry
mountain
forest
livingroom
street
livingroom
mountain
office
forest
kitchen
street
suburb
street
office
suburb
opencountry
livingroom
street
insidecity
office
bedroom
highway
coast
tallbuilding
tallbuilding
kitchen
mountain
opencountry
opencountry
bedroom
office
livingroom
forest
livingroom
bedroom
coast
street
livingroom
office
stre

In [92]:
predict_digit=np.argmax((pred2+pred3+pred4),axis=1)
i = 0
#for i in predict_digit:
#    print(get_classlabel(i))

In [93]:
del predict_digit

In [38]:
pred1
np.save('InceptionV3', pred1)

In [94]:
np.save('predall', predall)


In [95]:
del model
from keras import backend as K 
K.clear_session()

In [96]:
gc.collect()

1455605